# Bias and Disparity Detection Engine Demo: Open Sentencing

The purpose of this notebook is to demonstrate various capabilities of the bias and disparity detection engine (BDDE) in aiding the [Open Sentencing](https://github.com/Call-for-Code-for-Racial-Justice/Open-Sentencing) solution. The goal of Open Sentencing is to reduce racial disparities in the U.S. criminal justice system and the BDDE forms one part of the solution by highlighting disparities in historical data.

Below, we see a diagram of the steps in the criminal justice system. This demonstrates the multitude of possible outcomes and paths an accused individual may take and shows the scope of the challenge of reducing bias which may enter the system at any point in the process.

<img src="https://www.bjs.gov/content/downloadchart.cfm">

In [1]:
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from server.routes.engine import *

The specified config.json file /Users/shoffman/Projects/External/embrace-call-for-code/bias-detection-engine/notebooks/server/config/mappings.json does not exist


## The Data

The data we are analyzing is composed of three major parts:

* the first part relates to a simplified version of the diagram above and contains information about the path each defendant took through the criminal justice system and the eventual outcome type they received,

* the second part contains more detailed sentencing outcomes for those who were given prison sentences along with the information about the case used to determine sentencing guidelines according to the [USSC](https://www.ussc.gov/),

* and the third part is a structured representation of the facts and evidence surrounding the case.

Currently, we focus on two offenses, drug trafficking and drug possession, specifically marijuana. Also, the data used here are not real cases but simulated to preserve anonymity. Please refer to [this notebook](https://github.com/Call-for-Code-for-Racial-Justice/bias-detection-engine/blob/master/notebooks/weighted_drug_trafficking_data.ipynb) for a description of how the data were generated.

The cell below loads the data and shows a preview of the data on the stages encountered by each defendant.

In [2]:
df = pd.read_csv('../data/simulated_data_v0.4.csv', index_col=0).set_index(['Case ID', 'Race', 'Gender', 'Citizenship'])
fp = pd.concat([df.pop(c) for c in df.columns if c.startswith('FP')], axis=1)
sent = pd.concat([df.pop(c) for c in df.columns[-5:]], axis=1)
df

,,,,Alleged Crime,Alleged Crime Category,Alleged Crime Reported,Investigation,Arrest,Charges filed,Diversion,Court initial appearance,Preliminary hearing,Bail or detention hearing,Grand jury,Information stage,Arraignment,Plea bargain,Trial,Convicted,Sentencing
Case ID,Race,Gender,Citizenship,,,,,,,,,,,,,,,,,
10000,Other,Female,U.S. Citizen,Drug trafficking,Felony,Investigation,Unsolved or not arrested OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001,Hispanic,Male,U.S. Citizen,Drug trafficking,Felony,Investigation,Unsolved or not arrested OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,Black,Female,U.S. Citizen,Drug trafficking,Felony,Investigation,Unsolved or not arrested OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,White,Female,U.S. Citizen,Drug trafficking,Felony,Investigation,Arrest,Charges filed,Court initial appearance,NaN,Charges dropped or dismissed OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10004,White,Male,U.S. Citizen,Drug trafficking,Felony,Investigation,Arrest,Released OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70388,Other,Female,U.S. Citizen,Drug trafficking,Felony,Investigation,Arrest,Charges filed,Court initial appearance,NaN,Move to preliminary hearing,Bail or detention hearing,Move to collect information,NaN,Arraignment,Plea bargain,Convicted,NaN,Sentencing,Prison and alternatives
70389,Hispanic,Female,U.S. Citizen,Drug trafficking,Felony,Investigation,Unsolved or not arrested OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70390,White,Female,U.S. Citizen,Drug trafficking,Felony,Investigation,Arrest,Charges filed,Court initial appearance,NaN,Charges dropped or dismissed OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, we simply filter the data to show only one type of crime, drug trafficking.

In [3]:
subset = df['Alleged Crime'] == 'Drug trafficking'
df = df.loc[subset, 'Alleged Crime Reported':]
fp = fp.loc[subset]
sent = sent.loc[subset]

## Group Fairness

The "bias table" shown below highlights transitions in the criminal justice pipeline which show low or high [disparate impact scores](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.sklearn.metrics.disparate_impact_ratio.html). Disparate impact is simply the ratio of rates at which the transition occurs for one group of defendants over the rest:

$$DI = \frac{Pr(outcome | group A)}{Pr(outcome | \neg group A)}$$

For example, for all Black defendants that make it to the arrest stage, we determine what proportion of those are subsequently released and compare it to the respective proportion for all other races combined. The acceptable range for this ratio is generally between 0.8 and 1.25.

This metric simply shows disparities in outcomes without accounting for any other factors. This assumes that we should strive for equality in the distribution of outcomes given that all people are "created equal" which may involve looking outside the scope of the justice system alone.

In [4]:
bias_table(df, 'Race', 'defendants')

Closer to 1 is better. Disparate impact < 0.8 implies bias against the transition shown (red), > 1.25 implies bias for the transition (blue).

Using the sentencing part of the data, we can also analyze disparities in sentencing between races. We compute a value similar to disparate impact but using a difference of average sentence lengths for each race overall compared to the rest:

$$mean(sentence | group A) - mean(sentence | \neg group A)$$

For this analysis, we must remove cases which result in no prison or life terms as there is no way to numerically compare these cases.

In [5]:
subset = df['Sentencing'].str.contains('Prison', case=False, na=False)
sent = sent.loc[subset & (sent['Estimated Sentence'] != 'life') & (sent['Given Sentence'] != 'life')].astype(int)
mean_difference(sent['Estimated Sentence'], sent['Given Sentence'], 'Race')

,Sentencing bias
Race,
Black,9.22
Hispanic,6.21
Other,9.42
White,-5.41


Values > 0 represent an longer sentence length in months on average and values < 0, a shorter sentence.

## Individual Fairness -- Work in progress

Individual fairness is the simple concept that similar individuals should receive similar outcomes. This differs from group fairness where the population is divided into groups and these groups are expected to have equality. How we measure similarity is a key question and here we will use the final portion of the data, fact patterns.

Please note, this part of the data is a work in progress and results below may not be insightful.

In [6]:
consistency_table(fp, df)

Low numbers imply higher _inconsistency_ in outcomes (consistent results are not shown in the table above for brevity).